In [50]:
import pandas as pd
import numpy as np

df=pd.read_csv('cleaned_data.csv')
pd.DataFrame(df)
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50317 entries, 0 to 50316
Data columns (total 22 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   customer_id                   50317 non-null  object 
 1   age                           50317 non-null  int64  
 2   gender                        50317 non-null  object 
 3   region_category               50317 non-null  object 
 4   membership_category           50317 non-null  object 
 5   joining_date                  50317 non-null  object 
 6   joined_through_referral       50317 non-null  object 
 7   preferred_offer_types         50317 non-null  object 
 8   medium_of_operation           50317 non-null  object 
 9   internet_option               50317 non-null  object 
 10  days_since_last_login         50317 non-null  int64  
 11  avg_time_spent                50317 non-null  float64
 12  avg_transaction_value         50317 non-null  float64
 13  a

,customer_id,age,gender,region_category,membership_category,joining_date,joined_through_referral,preferred_offer_types,medium_of_operation,internet_option,...,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback,churn_risk_score,dataset
0,fffe4300490044003600300030003800,18,F,Village,Platinum Membership,2017-08-17,No,Gift Vouchers/Coupons,Unknown,Wi-Fi,...,230.228691,4.123106,781.75,Yes,Yes,No,Not Applicable,Products always in Stock,2.0,train
1,fffe4300490044003100390032003600,44,F,Town,No Membership,2016-11-11,Yes,Gift Vouchers/Coupons,Desktop,Wi-Fi,...,145.006896,4.690416,500.69,No,Yes,Yes,Solved in Follow-up,Poor Website,5.0,train
2,fffe43004900440036003000330031003600,37,M,City,No Membership,2016-10-29,Yes,Gift Vouchers/Coupons,Desktop,Mobile_Data,...,158.869632,2.449490,567.66,No,Yes,Yes,Unsolved,Poor Website,5.0,train
3,fffe43004900440031003900350030003600,31,F,City,No Membership,2017-09-12,No,Credit/Debit Card Offers,Smartphone,Mobile_Data,...,156.472554,4.000000,663.06,No,Yes,Yes,Solved,Poor Website,5.0,train
4,fffe43004900440036003300320035003300,13,M,City,Gold Membership,2016-01-08,No,Gift Vouchers/Coupons,Unknown,Wi-Fi,...,117.833654,4.898979,722.27,Yes,No,Yes,Unsolved,No reason specified,3.0,train


In [51]:
#drop_value='Unknown' for gender
df=df[df['gender']!= 'Unknown']

In [52]:
#drop_value='Unknown' for preferred offer types
df=df[df['preferred_offer_types']!= 'Unknown' ]


In [53]:
#making customer_id as unique row identifier index

if df['customer_id'].is_unique:

    df.set_index('customer_id', inplace=True)

    print("Successfully set 'customer_id' as index.")

else:

    print("Cannot set 'customer_id' as index, duplicates still present.")
 

Successfully set 'customer_id' as index.


In [54]:
print(df.columns)


Index(['age', 'gender', 'region_category', 'membership_category',
       'joining_date', 'joined_through_referral', 'preferred_offer_types',
       'medium_of_operation', 'internet_option', 'days_since_last_login',
       'avg_time_spent', 'avg_transaction_value', 'avg_frequency_login_days',
       'points_in_wallet', 'used_special_discount',
       'offer_application_preference', 'past_complaint', 'complaint_status',
       'feedback', 'churn_risk_score', 'dataset'],
      dtype='object')


In [55]:



membership_mapping = {
        'No Membership': 0,
        'Basic Membership': 1,
        'Silver Membership': 2,
        'Gold Membership': 3,
        'Platinum Membership': 4,
        'Premium Membership': 5
    }
df['membership_category'] = df['membership_category'].map(membership_mapping)

# Custom binary encoding for 'medium_of_operation'
df['medium_smartphone'] = df['medium_of_operation'].map({
    'Smartphone': 1,
    'Desktop': 0,
    'Both': 1,
    'Unknown': 0
})
df['medium_desktop'] = df['medium_of_operation'].map({
    'Smartphone': 0,
    'Desktop': 1,
    'Both': 1,
    'Unknown': 0
})
# Map churn_risk_score to new values
df['churn_risk_score'] = df['churn_risk_score'].map({
    0: 3,
    1: 1,
    2: 2,
    3: 4,
    4: 5,
    5: 6
})
# Drop 'past_complaint', 'joining_date', and 'dataset' as they are not needed
df.drop(['past_complaint', 'joining_date', 'dataset'], axis=1, inplace=True)
# Define categorical columns for One-Hot Encoding
categorical_columns = ['gender', 'region_category', 'joined_through_referral', 
                       'preferred_offer_types', 'internet_option', 
                       'used_special_discount', 'offer_application_preference', 
                       'complaint_status', 'feedback']

# Drop the original 'medium_of_operation' column after encoding
df.drop('medium_of_operation', axis=1, inplace=True)

In [56]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

# Create a ColumnTransformer for one-hot encoding and scaling
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first'), categorical_columns),
        ('scaler', StandardScaler(), ['age', 'days_since_last_login', 
                                      'avg_time_spent', 'avg_transaction_value', 
                                      'avg_frequency_login_days', 'points_in_wallet']),
    ],
    remainder='passthrough'  # Keep 'medium_smartphone', 'medium_desktop', 'membership_category'
)


In [57]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report


print(X.head())
# Define features (X) and target (y)
df = df.dropna(subset=['churn_risk_score'])
X = df.drop('churn_risk_score', axis=1)
y = df['churn_risk_score']

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



                                      age gender region_category  \
customer_id                                                        
fffe4300490044003600300030003800       18      F         Village   
fffe4300490044003100390032003600       44      F            Town   
fffe43004900440036003000330031003600   37      M            City   
fffe43004900440031003900350030003600   31      F            City   
fffe43004900440036003300320035003300   13      M            City   

                                      membership_category  \
customer_id                                                 
fffe4300490044003600300030003800                        4   
fffe4300490044003100390032003600                        0   
fffe43004900440036003000330031003600                    0   
fffe43004900440031003900350030003600                    0   
fffe43004900440036003300320035003300                    3   

                                     joined_through_referral  \
customer_id                    

In [58]:

# # Define the pipeline with the preprocessor and Logistic Regression
# pipeline = Pipeline(steps=[
#     ('preprocessor', preprocessor),
#     ('classifier', LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000))
# ])

# # Fit the model
# pipeline.fit(X_train, y_train)

# # Make predictions
# y_pred = pipeline.predict(X_test)

# # Evaluate the model
# print(classification_report(y_test, y_pred))

In [59]:
# Check the unique classes in y_train
print("Unique classes in y_train:", np.unique(y_train))


Unique classes in y_train: [1. 2. 3. 4. 5. 6.]


In [60]:
# from sklearn.preprocessing import PolynomialFeatures

# poly = PolynomialFeatures(degree=2, interaction_only=True)
# X_poly = poly.fit_transform(X_train)


In [61]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Ensure y_train does not contain NaN values
y_train = y_train.dropna()


# Define the pipeline with automatic class weights
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000, class_weight='balanced'))
])

# Fit the model
pipeline.fit(X_train, y_train)

# Make predictions
y_pred = pipeline.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

         1.0       0.58      0.63      0.60       492
         2.0       0.58      0.56      0.57       487
         3.0       0.02      0.05      0.03       195
         4.0       0.77      0.75      0.76      1860
         5.0       0.44      0.39      0.41      1789
         6.0       0.67      0.67      0.67      1676

    accuracy                           0.58      6499
   macro avg       0.51      0.51      0.51      6499
weighted avg       0.60      0.58      0.59      6499



In [46]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Ensure y_train does not contain NaN values
y_train = y_train.dropna()


# Define the pipeline with automatic class weights
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000, class_weight='balanced'))
])

# Fit the model
pipeline.fit(X_train, y_train)

# Make predictions
y_pred = pipeline.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))


ValueError: could not convert string to float: 'M'

ValueError: could not convert string to float: 'M'

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 49844 entries, fffe4300490044003600300030003800 to fffe43004900440036003200340030003100
Data columns (total 36 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   age                                             49844 non-null  int64  
 1   membership_category                             49844 non-null  int64  
 2   joining_date                                    49844 non-null  object 
 3   days_since_last_login                           49844 non-null  int64  
 4   avg_time_spent                                  49844 non-null  float64
 5   avg_transaction_value                           49844 non-null  float64
 6   avg_frequency_login_days                        49844 non-null  float64
 7   points_in_wallet                                49844 non-null  float64
 8   feedback                                        49844 non-n

In [25]:
df[df['region_category_Unknown']==1]

,age,membership_category,joining_date,days_since_last_login,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,feedback,churn_risk_score,...,internet_option_Wi-Fi,used_special_discount_No,used_special_discount_Yes,offer_application_preference_No,offer_application_preference_Yes,complaint_status_No Information Available,complaint_status_Not Applicable,complaint_status_Solved,complaint_status_Solved in Follow-up,complaint_status_Unsolved
customer_id,,,,,,,,,,,,,,,,,,,,,
fffe4300490044003800300035003800,42,0,2016-07-12,19,20.714970,211.080127,4.898979,568.08,Poor Product Quality,6.0,...,False,True,False,False,True,False,False,False,False,True
fffe43004900440035003800330038003500,11,3,2016-07-28,10,13.620940,245.242084,2.000000,836.08,User Friendly Website,1.0,...,True,False,True,True,False,False,False,False,True,False
fffe43004900440035003200330039003300,44,0,2016-08-23,13,16.837161,87.199255,0.000000,553.45,No reason specified,6.0,...,True,False,True,False,True,False,False,False,False,True
fffe43004900440031003300370038003600,10,1,2015-06-08,12,5.627611,159.487398,0.000000,656.67,Poor Website,5.0,...,False,True,False,False,True,False,True,False,False,False
fffe43004900440034003700390034003800,54,1,2015-08-07,13,15.959950,106.516008,2.645751,572.41,Poor Product Quality,6.0,...,False,True,False,False,True,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fffe43004900440031003500330037003600,13,2,2016-04-04,12,5.735852,128.863726,1.732051,798.20,Products always in Stock,NaN,...,False,False,True,True,False,False,False,False,False,True
fffe4300490044003300370035003800,62,2,2017-05-01,2,13.065986,312.939850,1.414214,774.27,Products always in Stock,NaN,...,True,True,False,False,True,False,True,False,False,False
fffe43004900440033003300340030003700,50,1,2017-10-28,21,6.559726,70.590580,3.741657,1032.33,Poor Customer Service,NaN,...,True,False,True,True,False,False,False,True,False,False


In [26]:
df.head()

,age,membership_category,joining_date,days_since_last_login,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,feedback,churn_risk_score,...,internet_option_Wi-Fi,used_special_discount_No,used_special_discount_Yes,offer_application_preference_No,offer_application_preference_Yes,complaint_status_No Information Available,complaint_status_Not Applicable,complaint_status_Solved,complaint_status_Solved in Follow-up,complaint_status_Unsolved
customer_id,,,,,,,,,,,,,,,,,,,,,
fffe4300490044003600300030003800,18,4,2017-08-17,17,17.338685,230.228691,4.123106,781.75,Products always in Stock,2.0,...,True,False,True,False,True,False,True,False,False,False
fffe4300490044003100390032003600,44,0,2016-11-11,14,22.719155,145.006896,4.690416,500.69,Poor Website,6.0,...,True,True,False,False,True,False,False,False,True,False
fffe43004900440036003000330031003600,37,0,2016-10-29,11,7.298630,158.869632,2.449490,567.66,Poor Website,6.0,...,False,True,False,False,True,False,False,False,False,True
fffe43004900440031003900350030003600,31,0,2017-09-12,20,10.636259,156.472554,4.000000,663.06,Poor Website,6.0,...,False,True,False,False,True,False,False,True,False,False
fffe43004900440036003300320035003300,13,3,2016-01-08,23,20.823544,117.833654,4.898979,722.27,No reason specified,4.0,...,True,False,True,True,False,False,False,False,False,True


In [27]:
# 7. Define the final pipeline with logistic regression
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000))
])


In [28]:

# 8. Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [31]:
# Check which columns might still be categorical strings
X_train



,age,gender,region_category,membership_category,joining_date,joined_through_referral,preferred_offer_types,internet_option,days_since_last_login,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,complaint_status,feedback,dataset,medium_smartphone,medium_desktop
customer_id,,,,,,,,,,,,,,,,,,,,
fffe43004900440035003200320033003800,16,M,City,0,2017-01-22,Yes,Without Offers,Mobile_Data,21,5.775812,157.813688,5.291503,565.58,Yes,No,Not Applicable,No reason specified,train,0,1
fffe43004900440035003200320032003900,28,M,Unknown,3,2016-12-13,Yes,Credit/Debit Card Offers,Wi-Fi,3,23.446748,228.733557,3.872983,749.26,No,Yes,No Information Available,Quality Customer Care,test,1,0
fffe43004900440034003000390035003600,42,M,Unknown,4,2017-11-17,No,Gift Vouchers/Coupons,Mobile_Data,13,14.699320,221.032351,4.582576,720.61,Yes,No,Not Applicable,Poor Product Quality,train,1,0
fffe43004900440031003000350034003400,10,F,Town,2,2016-04-19,No,Without Offers,Mobile_Data,10,9.875222,208.629121,4.472136,429.06,Yes,No,Not Applicable,No reason specified,test,1,0
fffe4300490044003900320034003900,62,M,Town,5,2016-02-11,Yes,Credit/Debit Card Offers,Fiber_Optic,4,15.276452,293.515230,2.000000,795.11,Yes,Yes,Not Applicable,Reasonable Price,test,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fffe43004900440033003500300033003700,20,M,Town,2,2015-12-05,No,Without Offers,Mobile_Data,8,20.535822,60.618232,6.244998,714.95,Yes,Yes,No Information Available,Poor Customer Service,train,1,0
fffe43004900440032003500390035003600,12,F,Town,0,2015-10-08,Yes,Without Offers,Mobile_Data,14,22.969545,62.972692,2.449490,568.37,No,Yes,Not Applicable,No reason specified,test,1,0
fffe43004900440031003100310034003400,57,F,City,1,2017-01-07,Yes,Credit/Debit Card Offers,Fiber_Optic,12,24.257988,153.912865,3.162278,649.48,Yes,No,Unsolved,No reason specified,test,1,0


In [29]:

# 9. Fit the pipeline on training data
pipeline.fit(X_train, y_train)


ValueError: could not convert string to float: 'M'

In [ ]:

# 10. Predict on test data
y_pred = pipeline.predict(X_test)


In [ ]:

# 11. Evaluate the model
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))
